In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load
model_input = pd.read_csv("../data/processed/model_input.csv")
print(f"Original features: {model_input.shape[1]}")

# INTERACTION FEATURES (differences matter more than absolute values)
model_input['win_rate_diff'] = model_input['home_win_rate'] - model_input['away_win_rate']
model_input['xg_diff'] = model_input['home_xg'] - model_input['away_xg']
model_input['shots_diff'] = model_input['home_shots'] - model_input['away_shots']
model_input['goal_diff_diff'] = model_input['home_goal_diff'] - model_input['away_goal_diff']
model_input['first_eff_diff'] = model_input['home_first_eff'] - model_input['away_first_eff']
model_input['second_eff_diff'] = model_input['home_second_eff'] - model_input['away_second_eff']
model_input['disparity_diff'] = model_input['home_offensive_disparity'] - model_input['away_offensive_disparity']

# RATIO FEATURES (relative strength)
model_input['xg_ratio'] = model_input['home_xg'] / (model_input['away_xg'] + 0.001)
model_input['shots_ratio'] = model_input['home_shots'] / (model_input['away_shots'] + 0.001)
model_input['first_eff_ratio'] = model_input['home_first_eff'] / (model_input['away_first_eff'] + 0.001)

# POLYNOMIAL FEATURES (capture non-linear relationships)
model_input['home_xg_squared'] = model_input['home_xg'] ** 2
model_input['away_xg_squared'] = model_input['away_xg'] ** 2
model_input['win_rate_diff_squared'] = model_input['win_rate_diff'] ** 2

# PRODUCT FEATURES (capture interactions)
model_input['home_xg_eff'] = model_input['home_xg'] * model_input['home_first_eff']
model_input['away_xg_eff'] = model_input['away_xg'] * model_input['away_first_eff']
model_input['home_strength'] = model_input['home_win_rate'] * model_input['home_xg'] * model_input['home_first_eff']
model_input['away_strength'] = model_input['away_win_rate'] * model_input['away_xg'] * model_input['away_first_eff']
model_input['strength_diff'] = model_input['home_strength'] - model_input['away_strength']

# AGGREGATED FEATURES (team quality indicators)
model_input['home_total_line_xg'] = model_input['home_first_xg'] + model_input['home_second_xg']
model_input['away_total_line_xg'] = model_input['away_first_xg'] + model_input['away_second_xg']
model_input['home_avg_line_eff'] = (model_input['home_first_eff'] + model_input['home_second_eff']) / 2
model_input['away_avg_line_eff'] = (model_input['away_first_eff'] + model_input['away_second_eff']) / 2

print(f"Enhanced features: {model_input.shape[1]}")
print(f"New features added: {model_input.shape[1] - 43}")

# Save enhanced dataset
model_input.to_csv("../data/processed/model_input_enhanced.csv", index=False)
print("\n✅ Saved: data/processed/model_input_enhanced.csv")


Original features: 43
Enhanced features: 65
New features added: 22

✅ Saved: data/processed/model_input_enhanced.csv


In [2]:
# Exclude leaky columns
exclude_cols = ['game_id', 'home_team', 'away_team', 'home_win', 'home_goals', 'away_goals', 
                'home_assists', 'away_assists']
feature_cols = [c for c in model_input.columns if c not in exclude_cols]

X = model_input[feature_cols].fillna(0)
y = model_input['home_win']

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Features: {len(feature_cols)}")
print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")


Features: 57
Train: (1049, 57)
Test: (263, 57)


In [3]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
import time

print("="*60)
print("TUNING XGBOOST")
print("="*60)

param_distributions = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [0.5, 1, 1.5, 2]
}

xgb_base = XGBClassifier(random_state=42, eval_metric='logloss')

start = time.time()
xgb_search = RandomizedSearchCV(
    xgb_base,
    param_distributions,
    n_iter=50,  # Try 50 random combinations
    cv=5,       # 5-fold cross-validation
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

xgb_search.fit(X_train, y_train)
elapsed = time.time() - start

best_xgb = xgb_search.best_estimator_
pred_xgb = best_xgb.predict_proba(X_test)[:, 1]
auc_xgb = roc_auc_score(y_test, pred_xgb)

print(f"\n✅ Best XGBoost AUC: {auc_xgb:.4f}")
print(f"Time: {elapsed/60:.1f} minutes")
print(f"\nBest parameters:")
for param, value in xgb_search.best_params_.items():
    print(f"  {param}: {value}")
print("="*60 + "\n")


TUNING XGBOOST
Fitting 5 folds for each of 50 candidates, totalling 250 fits

✅ Best XGBoost AUC: 0.6537
Time: 1.2 minutes

Best parameters:
  subsample: 0.7
  reg_lambda: 0.5
  reg_alpha: 0
  n_estimators: 100
  min_child_weight: 1
  max_depth: 4
  learning_rate: 0.01
  gamma: 0
  colsample_bytree: 0.6



In [4]:
from lightgbm import LGBMClassifier

print("="*60)
print("TUNING LIGHTGBM")
print("="*60)

param_distributions = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'num_leaves': [15, 31, 63, 127],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
    'min_child_samples': [10, 20, 30, 50],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [0, 0.01, 0.1, 1]
}

lgb_base = LGBMClassifier(random_state=42, verbose=-1)

start = time.time()
lgb_search = RandomizedSearchCV(
    lgb_base,
    param_distributions,
    n_iter=50,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

lgb_search.fit(X_train, y_train)
elapsed = time.time() - start

best_lgb = lgb_search.best_estimator_
pred_lgb = best_lgb.predict_proba(X_test)[:, 1]
auc_lgb = roc_auc_score(y_test, pred_lgb)

print(f"\n✅ Best LightGBM AUC: {auc_lgb:.4f}")
print(f"Time: {elapsed/60:.1f} minutes")
print(f"\nBest parameters:")
for param, value in lgb_search.best_params_.items():
    print(f"  {param}: {value}")
print("="*60 + "\n")


TUNING LIGHTGBM
Fitting 5 folds for each of 50 candidates, totalling 250 fits

✅ Best LightGBM AUC: 0.6489
Time: 0.5 minutes

Best parameters:
  subsample: 0.9
  reg_lambda: 0.1
  reg_alpha: 0
  num_leaves: 31
  n_estimators: 300
  min_child_samples: 50
  max_depth: 3
  learning_rate: 0.01
  colsample_bytree: 0.6



In [5]:
from catboost import CatBoostClassifier

print("="*60)
print("TUNING CATBOOST")
print("="*60)

param_distributions = {
    'iterations': [100, 200, 300, 500],
    'depth': [4, 5, 6, 7, 8],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 7],
    'border_count': [32, 64, 128],
    'random_strength': [0, 1, 2, 3]
}

cat_base = CatBoostClassifier(random_seed=42, verbose=False)

start = time.time()
cat_search = RandomizedSearchCV(
    cat_base,
    param_distributions,
    n_iter=50,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

cat_search.fit(X_train, y_train)
elapsed = time.time() - start

best_cat = cat_search.best_estimator_
pred_cat = best_cat.predict_proba(X_test)[:, 1]
auc_cat = roc_auc_score(y_test, pred_cat)

print(f"\n✅ Best CatBoost AUC: {auc_cat:.4f}")
print(f"Time: {elapsed/60:.1f} minutes")
print(f"\nBest parameters:")
for param, value in cat_search.best_params_.items():
    print(f"  {param}: {value}")
print("="*60 + "\n")


TUNING CATBOOST
Fitting 5 folds for each of 50 candidates, totalling 250 fits

✅ Best CatBoost AUC: 0.6838
Time: 4.0 minutes

Best parameters:
  random_strength: 1
  learning_rate: 0.03
  l2_leaf_reg: 1
  iterations: 200
  depth: 5
  border_count: 32



In [6]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

print("="*60)
print("BUILDING STACKING ENSEMBLE")
print("="*60)

# Base models (use best tuned models)
base_models = [
    ('xgb', best_xgb),
    ('lgb', best_lgb),
    ('cat', best_cat)
]

# Meta-learner (trains on base model predictions)
meta_learner = LogisticRegression(max_iter=1000, random_state=42)

# Stacking classifier
stacking = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_learner,
    cv=5,
    n_jobs=-1
)

print("Training stacking ensemble (this takes time)...")
start = time.time()
stacking.fit(X_train, y_train)
elapsed = time.time() - start

pred_stacking = stacking.predict_proba(X_test)[:, 1]
auc_stacking = roc_auc_score(y_test, pred_stacking)

print(f"\n✅ STACKING ENSEMBLE AUC: {auc_stacking:.4f}")
print(f"Time: {elapsed/60:.1f} minutes")
print("="*60 + "\n")


BUILDING STACKING ENSEMBLE
Training stacking ensemble (this takes time)...

✅ STACKING ENSEMBLE AUC: 0.6706
Time: 0.1 minutes



In [7]:
print("\n" + "="*60)
print("FINAL MODEL COMPARISON")
print("="*60)

results = pd.DataFrame({
    'Model': ['Baseline Logistic', 'Tuned XGBoost', 'Tuned LightGBM', 'Tuned CatBoost', 'Stacking Ensemble'],
    'AUC': [0.6900, auc_xgb, auc_lgb, auc_cat, auc_stacking],
    'Features': ['28 (original)', f'{len(feature_cols)} (enhanced)', f'{len(feature_cols)} (enhanced)', 
                 f'{len(feature_cols)} (enhanced)', f'{len(feature_cols)} (enhanced)']
}).sort_values('AUC', ascending=False)

results['Improvement'] = results['AUC'] - 0.6900
results['Rank'] = range(1, len(results) + 1)

print(results.to_string(index=False))
print("="*60)

best_model_name = results.iloc[0]['Model']
best_auc = results.iloc[0]['AUC']
improvement = results.iloc[0]['Improvement']

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   AUC: {best_auc:.4f}")
print(f"   Improvement over baseline: +{improvement:.4f} ({improvement/0.69*100:.1f}%)")



FINAL MODEL COMPARISON
            Model      AUC      Features  Improvement  Rank
Baseline Logistic 0.690000 28 (original)     0.000000     1
   Tuned CatBoost 0.683784 57 (enhanced)    -0.006216     2
Stacking Ensemble 0.670564 57 (enhanced)    -0.019436     3
    Tuned XGBoost 0.653702 57 (enhanced)    -0.036298     4
   Tuned LightGBM 0.648942 57 (enhanced)    -0.041058     5

🏆 BEST MODEL: Baseline Logistic
   AUC: 0.6900
   Improvement over baseline: +0.0000 (0.0%)


In [8]:
import pickle

# Determine best model
if auc_stacking >= max(auc_xgb, auc_lgb, auc_cat):
    final_model = stacking
    model_name = 'Stacking Ensemble'
elif auc_xgb >= max(auc_lgb, auc_cat):
    final_model = best_xgb
    model_name = 'XGBoost'
elif auc_lgb >= auc_cat:
    final_model = best_lgb
    model_name = 'LightGBM'
else:
    final_model = best_cat
    model_name = 'CatBoost'

print(f"Selected model: {model_name}")

# Save
with open('../outputs/final_optimized_model.pkl', 'wb') as f:
    pickle.dump({
        'model': final_model,
        'scaler': scaler,
        'feature_cols': feature_cols,
        'auc': best_auc,
        'model_name': model_name
    }, f)

print(f"✅ Saved: outputs/final_optimized_model.pkl")


Selected model: CatBoost
✅ Saved: outputs/final_optimized_model.pkl


In [9]:
# Load matchups
matchups = pd.read_excel("../data/raw/WHSDSC_Rnd1_matchups.xlsx")

# Load original model_input to build team features
model_input_orig = pd.read_csv("../data/processed/model_input_enhanced.csv")

# Build team features (with ALL enhanced features)
all_home_cols = [c for c in model_input_orig.columns if c.startswith('home_') 
                 and c not in ['home_team', 'home_win', 'home_goals', 'home_assists']]

team_features_dict = {}
for team in model_input_orig['home_team'].unique():
    home_games = model_input_orig[model_input_orig['home_team'] == team]
    home_stats = home_games[all_home_cols].mean()
    
    away_games = model_input_orig[model_input_orig['away_team'] == team]
    away_cols = [c.replace('home_', 'away_') for c in all_home_cols]
    away_stats = away_games[away_cols].mean()
    
    combined_stats = {}
    for col in all_home_cols:
        base = col.replace('home_', '')
        combined_stats[base] = home_stats[col]
    
    team_features_dict[team] = combined_stats

# Generate predictions
predictions_optimized = []

print("Generating optimized predictions...\n")

for _, row in matchups.iterrows():
    game_id = row['game_id']
    home = row['home_team'].lower().strip()
    away = row['away_team'].lower().strip()
    
    home_feats = team_features_dict.get(home, {})
    away_feats = team_features_dict.get(away, {})
    
    # Build feature row with ALL enhanced features
    feature_dict = {}
    for col in feature_cols:
        if col.startswith('home_'):
            base = col.replace('home_', '')
            feature_dict[col] = home_feats.get(base, 0)
        elif col.startswith('away_'):
            base = col.replace('away_', '')
            feature_dict[col] = away_feats.get(base, 0)
        else:
            # Handle interaction features
            if col in ['win_rate_diff', 'xg_diff', 'shots_diff', 'goal_diff_diff',
                      'first_eff_diff', 'second_eff_diff', 'disparity_diff']:
                home_base = col.replace('_diff', '')
                feature_dict[col] = home_feats.get(home_base, 0) - away_feats.get(home_base, 0)
            elif '_ratio' in col:
                parts = col.replace('_ratio', '').split('_')
                home_val = home_feats.get('_'.join(parts), 0)
                away_val = away_feats.get('_'.join(parts), 0)
                feature_dict[col] = home_val / (away_val + 0.001)
            elif '_squared' in col:
                base = col.replace('_squared', '')
                if 'home' in base:
                    val = home_feats.get(base.replace('home_', ''), 0)
                elif 'away' in base:
                    val = away_feats.get(base.replace('away_', ''), 0)
                else:
                    val = feature_dict.get(base, 0)
                feature_dict[col] = val ** 2
            else:
                feature_dict[col] = 0
    
    X_game = pd.DataFrame([feature_dict])[feature_cols].fillna(0)
    X_game_scaled = scaler.transform(X_game)
    
    prob = final_model.predict_proba(X_game_scaled)[0, 1]
    
    predictions_optimized.append({
        'game_id': game_id,
        'home_win_prob': prob
    })
    
    print(f"{game_id}: {home:12} vs {away:12} → {prob:.1%}")

# Save
submission_optimized = pd.DataFrame(predictions_optimized)
submission_optimized.to_csv("../outputs/round1_predictions_optimized.csv", index=False)

print(f"\n{'='*60}")
print(f"✅ SAVED: outputs/round1_predictions_optimized.csv")
print(f"📊 Model: {model_name}")
print(f"📊 AUC: {best_auc:.4f}")
print(f"📊 Improvement: +{improvement:.4f}")
print(f"{'='*60}")


Generating optimized predictions...

game_1: brazil       vs kazakhstan   → 79.1%
game_2: netherlands  vs mongolia     → 73.7%
game_3: peru         vs rwanda       → 75.8%
game_4: thailand     vs oman         → 80.0%
game_5: pakistan     vs germany      → 72.3%
game_6: india        vs usa          → 61.9%
game_7: panama       vs switzerland  → 80.0%
game_8: iceland      vs canada       → 64.4%
game_9: china        vs france       → 74.6%
game_10: philippines  vs morocco      → 62.7%
game_11: ethiopia     vs saudi_arabia → 68.3%
game_12: singapore    vs new_zealand  → 71.6%
game_13: guatemala    vs south_korea  → 70.6%
game_14: uk           vs mexico       → 69.5%
game_15: vietnam      vs serbia       → 57.9%
game_16: indonesia    vs uae          → 61.9%

✅ SAVED: outputs/round1_predictions_optimized.csv
📊 Model: CatBoost
📊 AUC: 0.6900
📊 Improvement: +0.0000


In [10]:
print("CROSS-VALIDATION SCORES:")
print("="*60)
print(f"XGBoost CV AUC: {xgb_search.best_score_:.4f}")
print(f"LightGBM CV AUC: {lgb_search.best_score_:.4f}")
print(f"CatBoost CV AUC: {cat_search.best_score_:.4f}")
print("="*60)


CROSS-VALIDATION SCORES:
XGBoost CV AUC: 0.6832
LightGBM CV AUC: 0.6799
CatBoost CV AUC: 0.6875


In [11]:
import pandas as pd
import pickle
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix

# Load baseline
model = pickle.load(open("../outputs/baseline_model.pkl", "rb"))
scaler = pickle.load(open("../outputs/baseline_scaler.pkl", "rb"))
model_input = pd.read_csv("../data/processed/model_input.csv")

# Get baseline features
feature_cols = list(scaler.feature_names_in_)
X = model_input[feature_cols].fillna(0)
y = model_input['home_win']

# Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_test_scaled = scaler.transform(X_test)

# Predictions
probs = model.predict_proba(X_test_scaled)[:, 1]
preds = (probs > 0.5).astype(int)

# Analyze errors
test_df = model_input.loc[X_test.index].copy()
test_df['predicted_prob'] = probs
test_df['predicted'] = preds
test_df['actual'] = y_test.values
test_df['correct'] = (test_df['predicted'] == test_df['actual'])
test_df['error_magnitude'] = np.abs(test_df['predicted_prob'] - test_df['actual'])

# Find patterns in errors
print("ERROR ANALYSIS:")
print("="*60)
print(f"Total test samples: {len(test_df)}")
print(f"Correct predictions: {test_df['correct'].sum()} ({test_df['correct'].mean()*100:.1f}%)")
print(f"Incorrect predictions: {(~test_df['correct']).sum()} ({(~test_df['correct']).mean()*100:.1f}%)")

# High-confidence errors (model was sure but wrong)
high_conf_errors = test_df[(~test_df['correct']) & ((test_df['predicted_prob'] > 0.7) | (test_df['predicted_prob'] < 0.3))]
print(f"\nHigh-confidence errors: {len(high_conf_errors)}")

if len(high_conf_errors) > 0:
    print("\nTop 5 high-confidence errors:")
    print(high_conf_errors.nlargest(5, 'error_magnitude')[['home_team', 'away_team', 'predicted_prob', 'actual']].to_string())

# Close games (hardest to predict)
close_games = test_df[test_df['predicted_prob'].between(0.4, 0.6)]
print(f"\nClose predictions (40-60%): {len(close_games)}")
print(f"Accuracy on close games: {close_games['correct'].mean()*100:.1f}%")

# Mismatches (strong vs weak teams)
test_df['win_rate_diff'] = test_df['home_win_rate'] - test_df['away_win_rate']
mismatches = test_df[test_df['win_rate_diff'].abs() > 0.2]
print(f"\nMismatches (>20% win rate diff): {len(mismatches)}")
print(f"Accuracy on mismatches: {mismatches['correct'].mean()*100:.1f}%")

balanced = test_df[test_df['win_rate_diff'].abs() < 0.1]
print(f"\nBalanced games (<10% win rate diff): {len(balanced)}")
print(f"Accuracy on balanced: {balanced['correct'].mean()*100:.1f}%")

print("="*60)


ERROR ANALYSIS:
Total test samples: 263
Correct predictions: 173 (65.8%)
Incorrect predictions: 90 (34.2%)

High-confidence errors: 24

Top 5 high-confidence errors:
        home_team    away_team  predicted_prob  actual
566        brazil       mexico        0.876216       0
700      pakistan           uk        0.827159       0
1292     mongolia       mexico        0.181920       1
79        iceland  netherlands        0.182505       1
1     switzerland   kazakhstan        0.184776       1

Close predictions (40-60%): 87
Accuracy on close games: 60.9%

Mismatches (>20% win rate diff): 17
Accuracy on mismatches: 70.6%

Balanced games (<10% win rate diff): 162
Accuracy on balanced: 63.0%
